# Wprowadzenie do Spark SQL

## DataFrame w Spark SQL

DataFrame to abstrakcyjna struktura danych w Spark SQL, która reprezentuje tabelę danych z kolumnami i wierszami.
DataFrame jest rozszerzeniem RDD, które jest zbiorem obiektów RDD, w którym każdy obiekt jest strukturą danych zawierającą nazwane kolumny (schemat danych).

DataFrame można tworzyć na różne sposoby:
- z obiektu RDD
- z pliku CSV, JSON, Parquet
- z bazy danych
- z innych źródeł danych
- z innych DataFrame
- z kolekcji danych
- z danych strumieniowych
- z danych zewnętrznych
- z danych z API
 

## Inicjowanie Sparka
Pracę ze Sparkiem zaczynamy od zainicjowania sesji Sparka.
Aby zainicjować Sparka, musimy zaimportować pakiet `findspark` i uruchomić metodę `init()`:

In [1]:
import findspark
findspark.init() 

Następnie tworzymy obiekt sesji Sparka. Zwróć uwagę na ustawienie nazwy aplikacji Sparka: 

In [106]:
from pyspark.sql import SparkSession

In [107]:
# obiekt sesji zwykle ma nazwę "spark"
spark = SparkSession.builder.appName("DataScience").getOrCreate() 
spark

## Tworzenie obiektów DataFrame

W pierwszym ćwiczeniu spróbujemy stworzyć obiekty DataFrame z kolekcji danych, obiektu RDD, pliku CSV oraz pliku Parquet.


### Utworzenie obiektu DataFrame z obiektu RDD

In [110]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
# musimy zmienić obiekt RDD na obiekt RDD zawierający krotki, ponieważ obiekt RDD musi zawierać strukturę danych
# nie może zawierać pojedynczych wartości
rdd = rdd.map(lambda x: (x, ))
rdd.collect()

[(1,), (2,), (3,), (4,), (5,)]

Mając obiekt RDD, możemy zmienić go na obiekt DataFrame za pomocą metody `.toDF()`.

In [111]:
df = rdd.toDF()
df

DataFrame[_1: bigint]

In [112]:
# funkcja show() wyświetla zawartość obiektu DataFrame
df.show()

+---+
| _1|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+


In [27]:
# funkcja printSchema() wyświetla schemat danych obiektu DataFrame
df.printSchema()

root
 |-- _1: long (nullable = true)


In [29]:
# funkcja describe() wyświetla statystyki opisowe obiektu DataFrame
df.describe().show()

+-------+------------------+
|summary|                _1|
+-------+------------------+
|  count|                 5|
|   mean|               3.0|
| stddev|1.5811388300841898|
|    min|                 1|
|    max|                 5|
+-------+------------------+


Jak widać powyżej, obiekt DataFrame zawiera kolumnę z nazwą `_1`. Musimy zmienić nazwę kolumny na bardziej opisową.

In [113]:
df = rdd.toDF(["value"])
df.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
+-----+


Typ danych w obiekcie DataFrame jest automatycznie ustawiany na podstawie danych wejściowych. Możemy sprawdzić typ danych za pomocą metody `dtypes`.

In [114]:
df.dtypes

[('value', 'bigint')]

Jak widać powyżej, typ danych w kolumnie `value` został ustawiony na `int`. Możemy zobaczyć schemat danych za pomocą metody `schema`.

In [115]:
df.schema

StructType([StructField('value', LongType(), True)])

In [34]:
df.schema.names


['value']

In [35]:
df.schema.fields

[StructField('value', LongType(), True)]

### Utworzenie obiektu DataFrame z kolekcji danych

Użyjemy metody `createDataFrame()` do utworzenia obiektu DataFrame z kolekcji danych.
Wartości w kolumnach obiektu DataFrame mogą być różnych typów danych.

In [116]:
data = [("John", 25), ("Anna", 23), ("Mike", 30), ("Jane", 22)]

In [117]:
df = spark.createDataFrame(data, ["name", "age"])
df

DataFrame[name: string, age: bigint]

In [118]:
df.show()

+----+---+
|name|age|
+----+---+
|John| 25|
|Anna| 23|
|Mike| 30|
|Jane| 22|
+----+---+


### Utworzenie obiektu DataFrame z pliku CSV

DataFrame można również utworzyć z pliku CSV. W tym celu używamy metody `read.csv()`.


In [120]:
df = spark.read.csv("../../data/stock/stock.csv", header=True, inferSchema=True)
df

DataFrame[date: date, company: string, symbol: string, opening_price: double, closing_price: double, volume: int]

In [121]:
df.show(5)

+----------+----------------+------+-------------+-----------------+------+
|      date|         company|symbol|opening_price|    closing_price|volume|
+----------+----------------+------+-------------+-----------------+------+
|2024-04-01|    Example Corp|   EXC|       114.91|           111.05|  1780|
|2024-04-02|      Another Co|   ANC|        74.97|             76.2|  3478|
|2024-04-03|Tech Innovations|   TEI|        90.52|94.33999999999999|  3035|
|2024-04-04|     Health Plus|   HPL|        69.35|69.61999999999999|  2756|
|2024-04-05|   Energy United|   ENU|        29.78|            28.75|  2445|
+----------+----------------+------+-------------+-----------------+------+


### Utworzenie obiektu DataFrame z pliku Parquet

Parquet to format pliku binarnego używany w Sparku do przechowywania danych.

In [123]:
df = spark.read.parquet("../../data/sklep/categories")
df

DataFrame[category_id: int, category_department_id: int, category_name: string]

## Podstawowe akcje na obiektach DataFrame
Akcja to operacja, która zwraca wartość do sterownika programu po przetworzeniu danych.

In [50]:
df = spark.read.parquet("../../data/sklep/categories")

### show()

`show()` wyświetla zawartość obiektu DataFrame.

In [51]:
df.show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+


In [52]:
df.show(5, truncate=False)  # truncate=False wyświetla całą zawartość kolumny

+-----------+----------------------+-------------------+
|category_id|category_department_id|category_name      |
+-----------+----------------------+-------------------+
|1          |2                     |Football           |
|2          |2                     |Soccer             |
|3          |2                     |Baseball & Softball|
|4          |2                     |Basketball         |
|5          |2                     |Lacrosse           |
+-----------+----------------------+-------------------+


### printSchema()

`printSchema()` wyświetla schemat danych obiektu DataFrame.

In [54]:
df.printSchema()    # wyświetla schemat danych obiektu DataFrame

root
 |-- category_id: integer (nullable = true)
 |-- category_department_id: integer (nullable = true)
 |-- category_name: string (nullable = true)


### describe()

`describe()` wyświetla statystyki opisowe obiektu DataFrame.

In [55]:
df.describe().show()

+-------+------------------+----------------------+--------------+
|summary|       category_id|category_department_id| category_name|
+-------+------------------+----------------------+--------------+
|  count|               116|                   116|           116|
|   mean|              29.5|     5.172413793103448|          NULL|
| stddev|16.813297015212072|     2.035664028267438|          NULL|
|    min|                 1|                     2|   Accessories|
|    max|                58|                     8|Yoga & Pilates|
+-------+------------------+----------------------+--------------+


In [56]:
df.describe("category_id").show()

+-------+------------------+
|summary|       category_id|
+-------+------------------+
|  count|               116|
|   mean|              29.5|
| stddev|16.813297015212072|
|    min|                 1|
|    max|                58|
+-------+------------------+


In [57]:
df.describe("category_id", "category_name").show()

+-------+------------------+--------------+
|summary|       category_id| category_name|
+-------+------------------+--------------+
|  count|               116|           116|
|   mean|              29.5|          NULL|
| stddev|16.813297015212072|          NULL|
|    min|                 1|   Accessories|
|    max|                58|Yoga & Pilates|
+-------+------------------+--------------+


### count()

`count()` zwraca liczbę wierszy obiektu DataFrame.

In [58]:
df.count()

116

### select()

`select()` zwraca nowy obiekt DataFrame z wybranymi kolumnami.

In [124]:
df.select("*").show()

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
|          6|                     2|   Tennis & Racquet|
|          7|                     2|             Hockey|
|          8|                     2|        More Sports|
|          9|                     3|   Cardio Equipment|
|         10|                     3|  Strength Training|
|         11|                     3|Fitness Accessories|
|         12|                     3|       Boxing & MMA|
|         13|                     3|        Electronics|
|         14|                     3|     Yoga & Pilates|
|         15|                  

In [66]:
df.select("category_id", "category_name").show(5)       # wybiera kolumny category_id i category

+-----------+-------------------+
|category_id|      category_name|
+-----------+-------------------+
|          1|           Football|
|          2|             Soccer|
|          3|Baseball & Softball|
|          4|         Basketball|
|          5|           Lacrosse|
+-----------+-------------------+


In [65]:
df.select(df["category_id"], df["category_name"]).show(5)       # można odwoływać się do kolumn za pomocą nawiasów kwadratowych

+-----------+-------------------+
|category_id|      category_name|
+-----------+-------------------+
|          1|           Football|
|          2|             Soccer|
|          3|Baseball & Softball|
|          4|         Basketball|
|          5|           Lacrosse|
+-----------+-------------------+


In [125]:
df.select(df.category_id, df.category_name).show(5)     # można odwoływać się do kolumn za pomocą kropki

+-----------+-------------------+
|category_id|      category_name|
+-----------+-------------------+
|          1|           Football|
|          2|             Soccer|
|          3|Baseball & Softball|
|          4|         Basketball|
|          5|           Lacrosse|
+-----------+-------------------+


### filter()

`filter()` zwraca nowy obiekt DataFrame z wierszami spełniającymi warunek.


In [136]:
df2 = df.filter(df["category_id"] == 1)
df2.show()

+-----------+----------------------+-------------+
|category_id|category_department_id|category_name|
+-----------+----------------------+-------------+
|          1|                     2|     Football|
|          1|                     2|     Football|
+-----------+----------------------+-------------+


In [68]:
df.filter(df["category_id"] == 1).select("category_id", "category_name").show()

+-----------+-------------+
|category_id|category_name|
+-----------+-------------+
|          1|     Football|
|          1|     Football|
+-----------+-------------+


In [137]:
df.filter(df["category_id"] == 1).filter(df["category_name"] == "Football").show()

+-----------+----------------------+-------------+
|category_id|category_department_id|category_name|
+-----------+----------------------+-------------+
|          1|                     2|     Football|
|          1|                     2|     Football|
+-----------+----------------------+-------------+


### distinct()

`distinct()` zwraca obiekt DataFrame bez duplikatów.


In [142]:
df.select("category_department_id").distinct().show()

+----------------------+
|category_department_id|
+----------------------+
|                     6|
|                     3|
|                     5|
|                     4|
|                     8|
|                     7|
|                     2|
+----------------------+


### orderBy()

`orderBy()` sortuje wiersze obiektu DataFrame według kolumny.

In [73]:
df.orderBy("category_id").show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
+-----------+----------------------+-------------------+


In [74]:
df.orderBy(df["category_id"].desc()).show(5)        # sortowanie malejące według kolumny category_id

+-----------+----------------------+--------------+
|category_id|category_department_id| category_name|
+-----------+----------------------+--------------+
|         58|                     8|   NFL Players|
|         58|                     8|   NFL Players|
|         57|                     8|   MLB Players|
|         57|                     8|   MLB Players|
|         56|                     8|World Cup Shop|
+-----------+----------------------+--------------+


In [75]:
df.orderBy(df["category_id"].asc(), df["category_name"].desc()).show(5)  # sortowanie rosnące według kolumny category_id oraz malejące według kolumny category_name

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
+-----------+----------------------+-------------------+


### groupBy()

`groupBy()` grupuje wiersze obiektu DataFrame według kolumny.

In [77]:
df.groupBy("category_id").count().show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

+-----------+-----+
|category_id|count|
+-----------+-----+
|         31|    2|
|         53|    2|
|         34|    2|
|         28|    2|
|         26|    2|
+-----------+-----+


In [148]:
df.groupBy("category_id").count().show(5)

+-----------+-----+
|category_id|count|
+-----------+-----+
|         31|    2|
|         53|    2|
|         34|    2|
|         28|    2|
|         26|    2|
+-----------+-----+


In [78]:
df.groupBy("category_id").agg({"category_name": "count"}).show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

+-----------+--------------------+
|category_id|count(category_name)|
+-----------+--------------------+
|         31|                   2|
|         53|                   2|
|         34|                   2|
|         28|                   2|
|         26|                   2|
+-----------+--------------------+


In [79]:
df.groupBy("category_id").agg({"category_name": "count", "category_id": "sum"}).show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

+-----------+--------------------+----------------+
|category_id|count(category_name)|sum(category_id)|
+-----------+--------------------+----------------+
|         31|                   2|              62|
|         53|                   2|             106|
|         34|                   2|              68|
|         28|                   2|              56|
|         26|                   2|              52|
+-----------+--------------------+----------------+


In [80]:
df.groupBy("category_id").agg({"category_name": "count", "category_id": "sum"}).orderBy("category_id").show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

+-----------+--------------------+----------------+
|category_id|count(category_name)|sum(category_id)|
+-----------+--------------------+----------------+
|          1|                   2|               2|
|          2|                   2|               4|
|          3|                   2|               6|
|          4|                   2|               8|
|          5|                   2|              10|
+-----------+--------------------+----------------+


### join()

`join()` łączy dwa obiekty DataFrame na podstawie kolumny.

Wykorzystamy dwa obiekty DataFrame: `categories` oraz `products`.


In [81]:
categories = spark.read.parquet("../../data/sklep/categories")
products = spark.read.parquet("../../data/sklep/products")


In [82]:
categories.show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+


In [83]:
products.show(5)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|                   |        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|                   |       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|                   |        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|                   |        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|                   |       199.99|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+


In [85]:
categories.join(products, categories["category_id"] == products["product_category_id"]).show(5)

+-----------+----------------------+-------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|category_id|category_department_id|category_name|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+-----------+----------------------+-------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|          2|                     2|       Soccer|         1|                  2|Quest Q64 10 FT. ...|                   |        59.98|http://images.acm...|
|          2|                     2|       Soccer|         1|                  2|Quest Q64 10 FT. ...|                   |        59.98|http://images.acm...|
|          2|                     2|       Soccer|         2|                  2|Under Armour Men'...|                   |       129.99|http://images.acm...|
|          2|                     2|       Soccer|  

Wyróżniamy kilka typów łączenia obiektów DataFrame:

- inner join - zwraca wiersze, które mają pasujące wartości w obu obiektach DataFrame
- outer join - zwraca wiersze, które mają pasujące wartości w jednym z obiektów DataFrame
- left join - zwraca wiersze, które mają pasujące wartości w lewym obiekcie DataFrame
- right join - zwraca wiersze, które mają pasujące wartości w prawym obiekcie DataFrame
- left semi join - zwraca wiersze z lewego obiektu DataFrame, które mają pasujące wartości w prawym obiekcie DataFrame
- left anti join - zwraca wiersze z lewego obiektu DataFrame, które nie mają pasujących wartości w prawym obiekcie DataFrame
- cross join - zwraca iloczyn kartezjański obu obiektów DataFrame
- natural join - zwraca wiersze, które mają pasujące wartości w obu obiektach DataFrame, ale bez powtarzających się kolumn

 
join_types = ["inner", "cross", "outer", "full", "left_outer", "right_outer"]


In [86]:
categories.join(products, categories["category_id"] == products["product_category_id"], "inner").show(5)

+-----------+----------------------+-------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|category_id|category_department_id|category_name|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+-----------+----------------------+-------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|          2|                     2|       Soccer|         1|                  2|Quest Q64 10 FT. ...|                   |        59.98|http://images.acm...|
|          2|                     2|       Soccer|         1|                  2|Quest Q64 10 FT. ...|                   |        59.98|http://images.acm...|
|          2|                     2|       Soccer|         2|                  2|Under Armour Men'...|                   |       129.99|http://images.acm...|
|          2|                     2|       Soccer|  

In [87]:
categories.join(products, categories["category_id"] == products["product_category_id"], "outer").show(5)

+-----------+----------------------+-------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|category_id|category_department_id|category_name|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+-----------+----------------------+-------------+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|          1|                     2|     Football|      NULL|               NULL|                NULL|               NULL|         NULL|                NULL|
|          1|                     2|     Football|      NULL|               NULL|                NULL|               NULL|         NULL|                NULL|
|          2|                     2|       Soccer|         1|                  2|Quest Q64 10 FT. ...|                   |        59.98|http://images.acm...|
|          2|                     2|       Soccer|  

### union()

`union()` łączy dwa obiekty DataFrame w jeden obiekt DataFrame.


In [89]:
df1 = spark.createDataFrame([(1, "John"), (2, "Anna")], ["id", "name"])
df2 = spark.createDataFrame([(3, "Mike"), (4, "Jane")], ["id", "name"])
df1.show()
df2.show()

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Anna|
+---+----+
+---+----+
| id|name|
+---+----+
|  3|Mike|
|  4|Jane|
+---+----+


In [90]:
df1.union(df2).show()

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Anna|
|  3|Mike|
|  4|Jane|
+---+----+


### withColumn()

`withColumn()` dodaje nową kolumnę do obiektu DataFrame.


In [91]:
df.withColumn("new_column", df["category_id"] + 1).show(5)

+-----------+----------------------+-------------------+----------+
|category_id|category_department_id|      category_name|new_column|
+-----------+----------------------+-------------------+----------+
|          1|                     2|           Football|         2|
|          2|                     2|             Soccer|         3|
|          3|                     2|Baseball & Softball|         4|
|          4|                     2|         Basketball|         5|
|          5|                     2|           Lacrosse|         6|
+-----------+----------------------+-------------------+----------+


In [92]:
df.withColumn("new_column", df["category_id"] + 1).select("category_id", "new_column").show(5)

+-----------+----------+
|category_id|new_column|
+-----------+----------+
|          1|         2|
|          2|         3|
|          3|         4|
|          4|         5|
|          5|         6|
+-----------+----------+


### drop()

`drop()` usuwa kolumnę z obiektu DataFrame.

In [93]:
df.drop("category_id").show(5)

+----------------------+-------------------+
|category_department_id|      category_name|
+----------------------+-------------------+
|                     2|           Football|
|                     2|             Soccer|
|                     2|Baseball & Softball|
|                     2|         Basketball|
|                     2|           Lacrosse|
+----------------------+-------------------+


### na()

`na()` zwraca obiekt DataFrame z wartościami brakującymi zastąpionymi wartościami.




In [95]:
# przykładowy obiekt DataFrame z wartościami brakującymi
df = spark.createDataFrame([(1, "John"), (2, None), (3, "Mike"), (4, "Jane")], ["id", "name"])
df.show()


+---+----+
| id|name|
+---+----+
|  1|John|
|  2|NULL|
|  3|Mike|
|  4|Jane|
+---+----+


In [96]:
df.na.fill("Anna").show()   # zastępuje wartości brakujące wartością "Anna"

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Anna|
|  3|Mike|
|  4|Jane|
+---+----+


In [97]:
df.na.fill({"name": "Anna"}).show()   # zastępuje wartości brakujące wartością "Anna" w kolumnie name

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Anna|
|  3|Mike|
|  4|Jane|
+---+----+


In [98]:
df.na.drop().show()   # usuwa wiersze z wartościami brakującymi

+---+----+
| id|name|
+---+----+
|  1|John|
|  3|Mike|
|  4|Jane|
+---+----+


### replace()

`replace()` zastępuje wartości w obiekcie DataFrame.
Zamiana wartości w obiekcie DataFrame jest możliwa tylko dla kolumn typu string.


In [135]:
df.replace("John", "Anna").show(5)   # zastępuje wartość "John" wartością "Anna"

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+


### cache()

`cache()` zapisuje obiekt DataFrame w pamięci podręcznej.

In [99]:
df = spark.createDataFrame([(1, "John"), (2, "Anna"), (3, "Mike"), (4, "Jane")], ["id", "name"])
df.cache()

DataFrame[id: bigint, name: string]

In [100]:
df.show()

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Anna|
|  3|Mike|
|  4|Jane|
+---+----+


### unpersist()

`unpersist()` usuwa obiekt DataFrame z pamięci podręcznej. Pozwala to na zwolnienie pamięci podręcznej.

In [126]:
df.unpersist()

DataFrame[category_id: int, category_department_id: int, category_name: string]

In [128]:
df.show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+


In [129]:
df.take(5)

[Row(category_id=1, category_department_id=2, category_name='Football'),
 Row(category_id=2, category_department_id=2, category_name='Soccer'),
 Row(category_id=3, category_department_id=2, category_name='Baseball & Softball'),
 Row(category_id=4, category_department_id=2, category_name='Basketball'),
 Row(category_id=5, category_department_id=2, category_name='Lacrosse')]

### collect(), take(), head(), first(), show(), toPandas()

`collect()` zwraca wszystkie wiersze obiektu DataFrame do sterownika programu.
`take(n)` zwraca n pierwszych wierszy obiektu DataFrame do sterownika programu.
`head(n)` zwraca n pierwszych wierszy obiektu DataFrame do sterownika programu.
`first()` zwraca pierwszy wiersz obiektu DataFrame do sterownika programu.
`show(n)` wyświetla n pierwszych wierszy obiektu DataFrame.
`toPandas()` zwraca obiekt DataFrame jako obiekt Pandas.


### write()

`write()` zapisuje obiekt DataFrame do pliku.

In [131]:
df = spark.read.parquet("../../data/sklep/categories")
df.write.csv("categories.csv")

## Zapytania SQL na obiektach DataFrame

Obiekty DataFrame można przekształcić na tabele tymczasowe, które można wykorzystać w zapytaniach SQL.

In [132]:
df = spark.read.parquet("../../data/sklep/categories")
df.createOrReplaceTempView("categories")


In [133]:
spark.sql("SELECT * FROM categories").show(5)

+-----------+----------------------+-------------------+
|category_id|category_department_id|      category_name|
+-----------+----------------------+-------------------+
|          1|                     2|           Football|
|          2|                     2|             Soccer|
|          3|                     2|Baseball & Softball|
|          4|                     2|         Basketball|
|          5|                     2|           Lacrosse|
+-----------+----------------------+-------------------+
